## A.

In [47]:
import pandas as pd
import matplotlib.pyplot as plt

riketAk9_path = "..\project\data\\riket2023_åk9_np.xlsx"

df_riketAk9 = pd.read_excel(riketAk9_path, skiprows=8)
df_riketAk9.head()

,Riket,Typ av huvudman,Totalt,Flickor,Pojkar,Totalt,Flickor,Pojkar,Totalt,Flickor,Pojkar
0,Riket,Samtliga,106941,51864,55077,97.0,97.1,97.0,15.7,15.8,15.7
1,Riket,Kommunal,84310,40675,43635,96.6,96.6,96.6,15.4,15.5,15.4
2,Riket,Enskild,22582,11165,11417,98.7,98.8,98.7,16.9,17.0,16.8
3,Riket,Skolverket,49,24,25,100.0,100.0,100.0,18.5,18.9,18.2


## B.

In [48]:
def reader(sheet, path):
    df = pd.read_excel(path, sheet_name=sheet, skiprows= 9)
    df.columns = ['Plats', 'Typ av huvudman', 'Totalt (A-F)', 'Flickor (A-F)', 'Pojkar (A-F)',
                  'Totalt (A-E)%', 'Flickor (A-E)%', 'Pojkar (A-E)%',
                  'Totalt (poäng)', 'Flickor (poäng)', 'Pojkar (poäng)']
    
    return df

df_math = reader("Matematik", riketAk9_path)
print("Matematik:")

df_math.head()

Matematik:


,Plats,Typ av huvudman,Totalt (A-F),Flickor (A-F),Pojkar (A-F),Totalt (A-E)%,Flickor (A-E)%,Pojkar (A-E)%,Totalt (poäng),Flickor (poäng),Pojkar (poäng)
0,Riket,Kommunal,84163,40386,43777,88.1,87.7,88.5,11.6,11.5,11.7
1,Riket,Enskild,22553,11137,11416,92.9,92.2,93.5,13.0,12.8,13.1
2,Riket,Skolverket,50,25,25,~100,~100,~100,14.1,13.5,14.7


In [49]:

df_math["Totalt (A-E)%"] = pd.to_numeric(df_math["Totalt (A-E)%"], errors='coerce')
df_math["Pojkar (A-E)%"] = pd.to_numeric(df_math["Pojkar (A-E)%"], errors='coerce')
df_math["Flickor (A-E)%"] = pd.to_numeric(df_math["Flickor (A-E)%"], errors='coerce')


df_math["Totalt (F)"] = df_math["Totalt (A-F)"] - (df_math["Totalt (A-F)"] * df_math["Totalt (A-E)%"] / 100)
df_math["Pojkar (F)"] = df_math["Pojkar (A-F)"] - (df_math["Pojkar (A-F)"] * df_math["Pojkar (A-E)%"] / 100)
df_math["Flickor (F)"] = df_math["Flickor (A-F)"] - (df_math["Flickor (A-F)"] * df_math["Flickor (A-E)%"] / 100)

print("Totalt antal studenter med F: ", df_math["Totalt (F)"].sum().round())
print("Totalt antal flickor med F: ", df_math["Flickor (F)"].sum().round())
print("Total antal pojkar med F: ", df_math["Pojkar (F)"].sum().round())



Totalt antal studenter med F:  11617.0
Totalt antal flickor med F:  5836.0
Total antal pojkar med F:  5776.0


## C.

In [50]:

df_swedish = reader("Svenska", riketAk9_path)
df_swedish_2 = reader("Svenska som andraspråk", riketAk9_path)
df_english = reader("Engelska", riketAk9_path)


total_points = [
    df_math["Totalt (poäng)"]
    df_swedish["Totalt (poäng)"]
    df_swedish_2["Totalt (poäng)"]
    df_english["Totalt (poäng)"]
]


sheet_names = list(df_riketAk9.keys())
print(df_swedish)

   Plats Typ av huvudman  Totalt (A-F)  Flickor (A-F)  Pojkar (A-F)  \
0  Riket        Kommunal         72771          35484         37287   
1  Riket         Enskild         20414          10297         10117   
2  Riket      Skolverket            40             19            21   

   Totalt (A-E)%  Flickor (A-E)%  Pojkar (A-E)%  Totalt (poäng)  \
0           95.7            97.6           93.8            13.3   
1           97.6            98.7           96.6            14.4   
2          100.0           100.0          100.0            15.0   

   Flickor (poäng)  Pojkar (poäng)  
0             14.3            12.4  
1             15.2            13.5  
2             15.9            14.2  
